In [45]:
import csv
import mediapipe as mp
import cv2
import time
import math
import numpy as np
from copy import copy, deepcopy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from playsound import playsound


In [46]:
# fig = plt.figure()
# ax1 = fig.add_subplot(1, 1, 1)
eye_diff_list = []
xx = []

cap = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)
width=1920
height = 1080
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
pTime = 0

mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
drawSpecs = mpDraw.DrawingSpec(thickness=1, circle_radius=1)
faceMesh = mpFaceMesh.FaceMesh(
    static_image_mode = False,
    max_num_faces = 1,
    refine_landmarks=True,
    min_detection_confidence=0.70,
    min_tracking_confidence = 0.70   # Will we miss a lot of information with higher confidence threshold??
)

In [47]:
def ear(x1, y1, x2, y2, x3, y3, x4, y4):
    a = math.sqrt(math.pow((x2-x1), 2) + math.pow((y2-y1), 2))
    b = math.sqrt(math.pow((x4-x3), 2) + math.pow((y4-y3), 2))
    return b,a

In [48]:
def get_face(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img_gray, 1.3, 5)
    try:
        x,y,w,h = faces[0]
        image = image[y:y+w+100, x:x+h+100]
    except:
        print("face not found")
    return image

# for (x,y,w,h) in faces:
    #     cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,0), 2)
    #     cropped_image = crop_image(img, x, y, w, h)
    #     cv2.imshow(cropped_image)

In [49]:
def distance_points(x1, y1, x2, y2):
    return math.sqrt(pow(x1-x2, 2) + pow(y2-y1,2) )

In [50]:
def abs_mov_cal(*args):
    # print("Arguemnts : ", args)
    # print("Type : ", type(args))
    args = list(args)
    args = [i*100 for i in args]
    ans = []
    i = 0
    while i < len(args):
        ans.append(int(min(1, (max(args[i]-args[i+1],0))) ))
        i += 2
    print(ans)
    return ans

In [53]:
# # we can n dimensional numpy array with indxes wise alternate to features
# n-dimensional numpy array, with each feature corresponding to each row.
# as each feature has different number of landmarks, so dtype=object( learn it)
# feature_indices = np.array([list(), list(), list(), list(), list(), list()], dtype=object);

feature_indices = [set(),
                   set(),
                   set(),
                   set(),
                   set(),
                   set(),
                   set()]

# LIST - feature_indices = [ [] [] [] [] [] [] ]
for s, t in mpFaceMesh.FACEMESH_RIGHT_EYE:          # right eye      0
    feature_indices[0].update([s, t])
for s, t in mpFaceMesh.FACEMESH_LEFT_EYE:           # left eye       1
    feature_indices[1].update([s, t])
for s, t in mpFaceMesh.FACEMESH_LIPS:               # lips           2
    feature_indices[2].update([s, t])
for s, t in mpFaceMesh.FACEMESH_RIGHT_EYEBROW:      # right eyebrow  3
    feature_indices[3].update([s, t])
for s, t in mpFaceMesh.FACEMESH_LEFT_EYEBROW:       # left eyebrow   4
    feature_indices[4].update([s, t])
for s, t in mpFaceMesh.FACEMESH_FACE_OVAL:          # oval           5
    feature_indices[5].update([s, t])

flat_feature_indices = [ j for i in feature_indices for j in i]
# facial_indices = []
le, re, lip, reb, leb, oval = len(feature_indices[0]), len(feature_indices[1]), len(feature_indices[2]), len(feature_indices[3]), len(feature_indices[4]),len(feature_indices[5])
facial = 478 - le + re + lip + reb + oval
feature_landmarks_count = [le, re, lip, reb, leb, oval, facial]

current_landmarks = np.array([np.zeros(feature_landmarks_count[0]),
                     np.zeros(feature_landmarks_count[1]),
                     np.zeros(feature_landmarks_count[2]),
                     np.zeros(feature_landmarks_count[3]),
                     np.zeros(feature_landmarks_count[4]),
                     np.zeros(feature_landmarks_count[5]),
                     np.zeros(feature_landmarks_count[6])], dtype=object)
current_landmarks = np.array(current_landmarks, dtype=object)
last_landmarks = np.array(np.copy(current_landmarks))

cl = [0]*len(feature_indices[0])
ll = [0]*len(feature_indices[0])

diff = 0
fail_count = 0
change_detected_count = 0

current_landmark_list = np.array([0 for i in range(478)])
last_landmark_list = np.array([0 for i in range(478)])

diff_list = [0] * 7
euclidean_distance_list = [0] * 7

last_euclidean_distance_list = [0] * 8
current_euclidean_distance_list = [0] * 8

eye_diff_list = []

current_facial_change = 0
last_facial_change = 0


blink_count = 0
current_eye_state = 2


allImages = []
videoOutput = cv2.VideoWriter("eyeVideo.avi", cv2.VideoWriter_fourcc(*'XVID'), 10, (640, 480))
eye_diff_list = []
xx = []

print("Current Landmarks : ", current_landmarks.shape)
print("Last Landmarks : ", last_landmarks.shape)
print("Feature indices : ", feature_indices)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#       HEAD different FEATURE CHANGE
current_changes = np.zeros(7) #  hh,hv,hh+hv,theta,lipsum,lipratio,eyeratio
past_changes = np.zeros(7)


#------------------------------------------------------------------------------------------------------------------------------------------------------------------------

curr_past = 0
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#       CSV DATA FILE OPENING

field_names = ["time", "left_eye", "right_eye","lips","head_h", "head_v", "head_slope", "planar_angle","abs_mov", "lip_sum","lip_ratio"]
state_string = "Open"
frame, e1, e2, l1, hh, hv, hslope, cosine_theta, abs_mov, lip_sum, lip_euclidean = 1,0,0,0,0,0,0,0,0,0,0
# frame, eye, eargraph, facial_change = 1,1,0,0
with open('data.csv', 'w') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)
    csv_writer.writeheader()
    info = {"time": frame, "left_eye": e1, "right_eye": e2, "lips": l1,  "head_h": hh, "head_v" : hv, "head_slope": hslope, "planar_angle": cosine_theta,"abs_mov":abs_mov, "lip_sum":lip_sum, "lip_ratio":l1}
    csv_writer.writerow(info)

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
i = -1
count = 0
head_movement_count = 0
while True:
    with open("data.csv", 'a') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)
        info = {"time": frame, "left_eye": e1, "right_eye": e2, "lips": l1,  "head_h": hh, "head_v": hv, "head_slope": hslope, "planar_angle":cosine_theta, "abs_mov":abs_mov, "lip_sum":lip_sum, "lip_ratio":l1}
        csv_writer.writerow(info)

    success, img = cap.read()

    img = cv2.flip(img, 1)

    # cropped_image = get_face(img)
    # cv2.imshow("",cropped_image)
    # img = cropped_image

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = imgRGB
    # kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    # img = cv2.filter2D(img, -1, kernel)
    # cv2.imshow("sad",img)

    img.flags.writeable = False
    # try:
    #     mp_get_face(img)
    # except:
    #     count += 1
    #     print("face not found : ", count)

    imgRGB = img
    # img = imgRGB

    img_size = img.shape

    results = faceMesh.process(imgRGB)
    # img = np.zeros(img.shape, np.uint8)

    facial_distance = 0

    if results.multi_face_landmarks:
            l = []
            for faceLms in results.multi_face_landmarks:
                mpDraw.draw_landmarks(img, faceLms,
                                      mpFaceMesh.FACEMESH_CONTOURS,
                                      landmark_drawing_spec=drawSpecs,
                                      connection_drawing_spec=drawSpecs)

                colors = [(0, 0, 0), (255, 255, 255), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),(0, 0, 0)]

                facial_indices = [ind for ind, x in enumerate(faceLms.landmark) if ind not in flat_feature_indices]
                feature_indices[-1] = set(facial_indices)

                angle_horizontal = 0
                angle_vertical = 0

                # facial_distance = 0

                for cid, feature in enumerate(feature_indices):
                    index = 0
                    color_ind = 0
                    # if cid == 2:
                    euclidean_distance = 0
                    point_count = 0
                    for ind, lm in enumerate(faceLms.landmark):
                        if ind in feature:
                            # Coordinaated of every LANDMARK on FACE with origin
                            x, y, z = int(img_size[1]*lm.x), int(img_size[0]*lm.y), lm.z

                            if cid == 6:
                                # img = cv2.circle(img, (x, y), 1, (0, 255, 100), 2)
                                facial_distance += pow(pow((x-0), 2) + pow((y-450), 2), 1/2)
                                # img = cv2.line(img, (x,y), (a_oval_landmark[0], a_oval_landmark[1]), (0,0,0), 1)

                            # SUM DISTANCE of evey LANDMARK on FACE with origin
                            euclidean_distance += pow(pow(x, 2) + pow(y, 2), 1/2)
                            # img = cv2.line(img, (0,0), (x,y), (255,255,0), 1)

                            # POINTS in middle of FACE to CALCUALTE the VERTICAL/HORIZONTAL movement
                            if ind == list(feature_indices[5])[3] or ind == list(feature_indices[5])[10]:

                                a_oval_landmark = [x, y, z]
                                # img = cv2.line(img, (x,y), (600,0), (0,255,0), 2)
                                # img = cv2.line(img, (x,y), (0,350), (255,255,1), 2)

                                # img = cv2.line(img, (x,y), (20,y), (0,255,255), 2)
                                # img = cv2.line(img, (x,y), (x,20), (255,0,150), 2)

                                img = cv2.circle(img, (x, y), 1, (0, 0, 255), 5)

                                angle_horizontal += pow(pow((x-20), 2) + pow((y-y), 2), 1/2)
                                angle_vertical += pow(pow((x-x), 2) + pow((y-20), 2), 1/2)



                            if curr_past == 0:
                                last_landmarks[cid][index] = abs(x) + abs(y) + abs(z)
                            elif curr_past % 1 == 0:
                                current_landmarks[cid][index] = abs(x) + abs(y) + abs(z)


                            index += 1
                            color_ind += 1
                            point_count += 1
            # print("INDICIES")
            # print(list(feature_indices[5])[3])
            # print(list(feature_indices[5])[10])
    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #      COORDINATES EXTRACTION

            #HEAD
            hx, hy = int(faceLms.landmark[10].x*img_size[1]), int(faceLms.landmark[10].y*img_size[0])

            #Thuddi
            tx, ty = int(faceLms.landmark[152].x*img_size[1]), int(faceLms.landmark[152].y*img_size[0])

            # LIPS
            # Coordinates
            lx1, ly1 = int(faceLms.landmark[61].x*img_size[1]), int(faceLms.landmark[61].y*img_size[0])
            lx2, ly2 = int(faceLms.landmark[291].x*img_size[1]), int(faceLms.landmark[291].y*img_size[0])
            lx3, ly3 = int(faceLms.landmark[0].x*img_size[1]), int(faceLms.landmark[0].y*img_size[0])
            lx4, ly4 = int(faceLms.landmark[17].x*img_size[1]), int(faceLms.landmark[17].y*img_size[0])

            # euclidean aspect ratio
            lipa, lipb = ear(lx1, ly1, lx2, ly2, lx3, ly3, lx4, ly4)
            lip_euclidean = lipa / lipb

            # lips horizontal and vertical distance (sum maybe)
            liphor = distance_points(lx1, ly1, lx2, ly2)
            lipver = distance_points(lx3, ly3, lx4, ly4)


            # LEFT EYE
            # Coordinates
            x1, y1 = int(faceLms.landmark[133].x*img_size[1]), int(faceLms.landmark[133].y*img_size[0])
            x2, y2 = int(faceLms.landmark[33].x*img_size[1]), int(faceLms.landmark[33].y*img_size[0])
            x4, y4 = int(faceLms.landmark[159].x*img_size[1]), int(faceLms.landmark[159].y*img_size[0])
            x3, y3 = int(faceLms.landmark[145].x*img_size[1]), int(faceLms.landmark[145].y*img_size[0])

            # euclidean aspect ratio
            eyea, eyeb = ear(x1,y1,x2,y2,x3,y3,x4,y4)
            eye_euclidean = eyea/eyeb

            # HEAD ANGULAR Movement Calculation
            cs = math.acos( math.sqrt( pow(tx-tx,2)+pow(hy-ty,2) )/math.sqrt( pow(hx-tx,2)+pow(hy-ty,2) ) )

    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#       LAST CURRENT UPDATION

            # INCREASING the WEIGHT of VERTIAL MOVEMENT (because vertial motion is not as much as horizontal change)
            # angle_vertical = angle_vertical * 3

            if curr_past ==  0:
                past_changes = angle_horizontal, angle_vertical, (angle_vertical+angle_horizontal), cs, liphor+lipver, lip_euclidean, eye_euclidean
            else:
                # current_changes = angle_horizontal, angle_vertical, (angle_vertical+angle_horizontal), cs, liphor+lipver, lip_euclidean, eye_euclidean
                if curr_past % 15 == 0:
                    current_changes[4] = liphor + lipver
                    current_changes[5] = lip_euclidean

                current_changes[0] = angle_horizontal
                current_changes[1] = angle_vertical
                current_changes[2] = (angle_vertical + angle_horizontal)
                current_changes[3] = cs
                current_changes[6] = eye_euclidean


    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #      DISTANCES CALCULATION for GRAPH

            for i in range(7):
                # diff_list[i] = (sum((abs(last_landmarks-current_landmarks))[i]))/feature_landmarks_count[i]
                diff_list[i] = (sum((np.power(abs(last_landmarks-current_landmarks), 1))[i]))/feature_landmarks_count[i]
                # diff_list[i] = (sum((np.power(abs(last_landmarks-current_landmarks), 1))[i]))

#       Time (graph's frame)
            frame = curr_past

#       Each Frame Absolute Distance
            # hh = angle_horizontal
            # hv = angle_vertical
            # hslope = angle_horizontal + angle_vertical
            # cosine_theta = pow(pow(hx-tx, 2) + pow(hy-ty, 2), 0.5) / pow( pow( tx-tx, 2)  +   pow(hy-ty , 2), 0.5  )
            # lip_sum = liphor + lipver

#       NOT ABSOLUTE CHANGE
            #e1 = eye_euclidean
            # e2 = 0
            # l1 = lip_euclidean
            # lip_sum = liphor + lipver
            # lip_sum = liphor + lipver
            # hh = current_changes[0] - past_changes[0]
            # hv = current_changes[1] - past_changes[1]
            # hslope = current_changes[2] - past_changes[2]
            # cosine_theta = current_changes[3] - past_changes[3]
            # abs_mov = ( 1 if max(hh-7,0)>0 else 0) | ( 1 if max(hv-7.5,0)>0 else 0) | (1 if max(cosine_theta-0.019,0)>0 else 0)

#       ABSOLUTE CHANGE
            hh, hv, hslope, cosine_theta, lip_sum, l1, e1 = np.abs(np.subtract(current_changes, past_changes))
            # abs_mov = ( 1 if max(hh-7,0)>0 else 0) | ( 1 if max(hv-7.5,0)>0 else 0) | (1 if max(cosine_theta-0.019,0)>0 else 0)

            absmov_list = list(abs_mov_cal(hh,7, hv,7.5, cosine_theta,0.19, lip_sum,10, l1,0.08, e1,0.065))
            abs_mov = np.bitwise_or.reduce(absmov_list)
            # print("!"*100)
            # print(min(1,max(cosine_theta-0.095, 0)))
            # print(min(1,max(hh-7, 0)))
            # print(min(1,max(hv-7.5, 0)))

        # cosine_theta = cs
            # hslope = 0 if abs(last_facial_change - current_facial_change) < 7 else 10
            '''
                    INCREASING the WEIGHT of VERTIAL MOVEMENT (because vertial motion is not as much as horizontal change)
                    2. If the face is parallely aligned with the camera then the detection is very goood.
                       else if the face is at an angle with the camera, titled backward or forward(resting position) then detections are not acurate as in parallel.
                    FALSE
                        -I had my method of calcualtion incorrect, now Coordinates are changed which calculate vertical&horizontal motion unbiased.
            '''


            # zoop = (current_euclidean_distance_list[5]/feature_landmarks_count[5]) - (last_euclidean_distance_list[5]/feature_landmarks_count[5])
            # eye = diff_list[0] if diff_list[0] > 1 else 1
            # facial_change = pow(5, facial_distance/1000)
            # facial_change = facial_distance
            # eargraph = angle_vertical + angle_horizontal
            # eargraph = abs(last_facial_change - current_facial_change)
            # eargraph = current_facial_change
            # eargraph = lip_euclidean
            # print(eargraph)
            # facial_change = (current_euclidean_distance_list[5]/feature_landmarks_count[5])/80
            # eargraph =  2 if euclidean < 0.29 else (if)
            # eargraph = current_eye_state

            #------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            #      STATE OF EYE
            if eye_euclidean > 0.29 and (current_eye_state == 0 or current_eye_state == 1):#fully open
                blink_count += 1
                current_eye_state = 2
                state_string = "FULL OPEN"
            elif eye_euclidean < 0.15 and (current_eye_state == 2 or current_eye_state == 1):
                blink_count += 1
                current_eye_state = 0
                state_string = "FULL CLOSED"
            elif 0.18 <= eye_euclidean < 0.29 and (current_eye_state == 0 or current_eye_state == 2):
                blink_count += 1
                current_eye_state = 1
                state_string = "HALF OPEN/CLOSED"

    #------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #      GRAPHICS on VIDEO STREAM

            # lip horizontal-vertial LINE
            img = cv2.line(img, (lx1, ly1), (lx2, ly2), (255,0,255), 2)
            img = cv2.line(img, (lx3, ly3), (lx4, ly4), (255,0,0), 2)

            # eye horizontal-vertial LINE
            img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 2)
            img = cv2.line(img, (x3,y3), (x4,y4), (0,0,255), 2)

            # cosine sides
            # img = cv2.line(img, (tx, hy), (tx,ty), (255, 255, 123), 3)
            # img = cv2.line(img, (hx, hy), (tx,ty), (255, 255, 123), 3)

            # forehead
            img = cv2.circle(img, (hx, hy), 2, (0,12,123), 6)

            # thuddi
            img = cv2.circle(img, (tx, ty), 2, (255, 55, 255), 5)

            #screen axes
            img = cv2.line(img, (20,0), (20,720), (255,255,255), 1)
            img = cv2.line(img, (0,20), (1200,20), (255,255,255), 1)

            # out_text = str(head_movement_count)
            # cv2.putText(img, out_text, (1000,50), cv2.FONT_HERSHEY_PLAIN, 2, (70,150,255), 2)
            cv2.putText(img, str("Lip Ratio: "+str(lip_euclidean)), (1000,115), cv2.FONT_HERSHEY_PLAIN, 2, (180,59,110), 2)
            cv2.putText(img, str("Lip Sum : " + str(lip_sum)), (1000, 140), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
            cv2.putText(img, f"{state_string}", (1000,85), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,0), 2)
            cv2.putText(img, f"eye ratio : {eye_euclidean}", (20, 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2 )

    #-------------------------------------------------------------------------------------------------------------------------

            # if(hslope):
            #     playsound("beep.mp3", False)
            #     head_movement_count += 1

            last_facial_change = current_facial_change
            last_landmarks = deepcopy(current_landmarks)
            past_changes = deepcopy(current_changes)

            if curr_past % 10 == 0:
                last_euclidean_distance_list = deepcopy(current_euclidean_distance_list)

            curr_past += 1

    else:
        # print("Result in results.multi_face_landmarks failed. Fail Count : ", fail_count);
        fail_count += 1
#-------------------------------------------------------------------------------------------------------------------------
#     FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f"fps: {int(fps)}", (20, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
#-------------------------------------------------------------------------------------------------------------------------

    cv2.imshow("image", img)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

Current Landmarks :  (7,)
Last Landmarks :  (7,)
Feature indices :  [{160, 33, 161, 163, 133, 7, 173, 144, 145, 246, 153, 154, 155, 157, 158, 159}, {384, 385, 386, 387, 388, 390, 263, 362, 398, 466, 373, 374, 249, 380, 381, 382}, {0, 267, 269, 270, 13, 14, 17, 402, 146, 405, 409, 415, 291, 37, 39, 40, 178, 308, 181, 310, 311, 312, 185, 314, 317, 318, 61, 191, 321, 324, 78, 80, 81, 82, 84, 87, 88, 91, 95, 375}, {65, 66, 70, 105, 107, 46, 52, 53, 55, 63}, {293, 295, 296, 300, 334, 336, 276, 282, 283, 285}, {132, 389, 136, 10, 397, 400, 148, 149, 150, 21, 152, 284, 288, 162, 297, 172, 176, 54, 58, 323, 67, 454, 332, 338, 93, 356, 103, 361, 234, 109, 365, 379, 377, 378, 251, 127}, set()]
[1, 1, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0]
[0,

KeyboardInterrupt: 

In [ ]:
  def mp_get_face(image):
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.3)
    mp_drawing = mp.solutions.drawing_utils

    result = face_detection.process(image)

    if result.detections:
        for detection in result.detections:
            mp_drawing.draw_detection(image, detection)
        # for fn, face in enumerate(result.detections):
        #     face_data = face.location_data
        #     # print("Confidene : ", round(face.score[0], 2))
        #     # print("Face Bounding Box : ", face_data.relative_bounding_box)
    else:
        print("Face not found")
    cv2.imshow("stream", image)

In [ ]:
import cv2
import mediapipe as mp
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image =  cap.read()
    image.flags.writeable = False
    cv2.imshow("first", image)
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.3)
    mp_drawing = mp.solutions.drawing_utils

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    result = face_detection.process(image)

    if result.detections:
        for detection in result.detections:
            mp_drawing.draw_detection(image, detection)
        # for fn, face in enumerate(result.detections):
        #     face_data = face.location_data
        #     # print("Confidene : ", round(face.score[0], 2))
        #     # print("Face Bounding Box : ", face_data.relative_bounding_box)
    else:
        print("Face not found")
    # cv2.imshow("stream", image)
    print("working")
    # cv2.imshow("ada", img)


In [ ]:
# i = -1
# while True:
#     with open("data.csv", 'a') as csv_file:
#         csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)
#         info = {"frame": frame, "eye": eye, "ear": eargraph, "facial": facial_change}
#         csv_writer.writerow(info)
#
#         success, img = cap.read()
#
#         img = cv2.flip(img, 1)
#         img.flags.writeable = False
#         imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img_size = img.shape
#
#         # videoOutput.write(img)
#         # allImages.append(img)
#         # print(allImages)
#
#
#         results = faceMesh.process(imgRGB)
#         img = np.zeros(img.shape, np.uint8)
#
#         # xcor1, xcor2 = 0, 0
#         # ycor1, ycor2 = 0, 0
#         # for i in range(54):
#         #     img = cv2.line(img, (xcor1, 0), (xcor2, 720), (0,255,0), 1)
#         #     img = cv2.line(img, (0, ycor1), (1280, ycor2), (0,255,0), 1)
#         #     xcor1 += 25
#         #     xcor2 += 25
#         #     ycor1 += 25
#         #     ycor2 += 25
#
#         facial_distance = 0
#
#         if results.multi_face_landmarks:
#             l = []
#             for faceLms in results.multi_face_landmarks:
#                 mpDraw.draw_landmarks(img, faceLms,
#                                       mpFaceMesh.FACEMESH_CONTOURS,
#                                       landmark_drawing_spec=drawSpecs,
#                                       connection_drawing_spec=drawSpecs)
#
#                 colors = [(0, 0, 0), (255, 255, 255), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),(0, 0, 0)]
#
#                 facial_indices = [ind for ind, x in enumerate(faceLms.landmark) if ind not in flat_feature_indices]
#                 feature_indices[-1] = set(facial_indices)
#
#                 # if i == 0:
#                 #     print("FEATURE INDICES ALL 7")
#                 #     for kkkk in feature_indices:
#                 #         print(kkkk)
#
#
#
#                 angle_horizontal = 0
#                 angle_vertical = 0
#
#                 # facial_distance = 0
#
#                 for cid, feature in enumerate(feature_indices):
#                     index = 0
#                     color_ind = 0
#                     # if cid == 2:
#                     euclidean_distance = 0
#                     point_count = 0
#                     for ind, lm in enumerate(faceLms.landmark):
#                         if ind in feature:
#                             x, y, z = int(img_size[1]*lm.x), int(img_size[0]*lm.y), lm.z
#
#                             if cid == 6:
#                                 img = cv2.circle(img, (x, y), 1, (0, 255, 100), 2)
#                                 facial_distance += pow(pow((x-0), 2) + pow((y-450), 2), 1/2)
#                                 # img = cv2.line(img, (x,y), (a_oval_landmark[0], a_oval_landmark[1]), (0,0,0), 1)
#
#                             # if point_count <= 5:
#                             euclidean_distance += pow(pow(x, 2) + pow(y, 2), 1/2)
#                             # img = cv2.line(img, (0,0), (x,y), (0,0,0), 1)
#                             # print(list(feature_indices[5])[1], type(list(feature_indices[5])[0]))
#                             # print("indices : ",ind, type(ind))
#                             # if ind == list(feature_indices[2])[6]:
#                             #     img = cv2.circle(img, (x, y), 1, (0,0,0), 10)
#                             #     print("Black : "+str(ind))
#                             # if ind == list(feature_indices[2])[12]:
#                             #     img = cv2.circle(img, (x, y), 1, (255,255,255), 10)
#                             #     print("White : "+ str(ind))
#                             # if ind == list(feature_indices[2])[24]:
#                             #     img = cv2.circle(img, (x, y), 1, (255,0,0), 10)
#                             #     print("blue : " + str(ind))
#                             # if ind == list(feature_indices[2])[0]:
#                             #     img = cv2.circle(img, (x, y), 1, (0,0,255), 10)
#                             #     print("red : "+ str(ind))
#
#                             if ind == list(feature_indices[5])[3] or ind == list(feature_indices[5])[10]:
#                                 # if ind == list(feature_indices[5])[3]:
#                                 a_oval_landmark = [x, y, z]
#                                 img = cv2.circle(img, (x, y), 1, (0, 0, 255), 5)
#                                 img = cv2.line(img, (x,y), (600,0), (255,0,0), 2)
#                                 img = cv2.line(img, (x,y), (0,350), (0,0,255), 2)
#
#                                 angle_vertical += pow(pow((x-600), 2) + pow((y-y), 2), 1/2)
#                                 angle_horizontal += pow(pow((x-x), 2) + pow((y-350), 2), 1/2)
#                             else:
#                                 pass
#                                 # img = cv2.circle(img, (x, y), 1, colors[1], 2)
#
#                             if curr_past == 0:
#                                 last_landmarks[cid][index] = abs(x) + abs(y) + abs(z)
#                             elif curr_past % 1 == 0:
#                                 current_landmarks[cid][index] = abs(x) + abs(y) + abs(z)
#
#
#                             index += 1
#                             color_ind += 1
#                             point_count += 1
#
#
#                 # a_oval_landmark = [feature_indices[5][0] ]
#                 # else:
#                 #     for ind, lm in enumerate(faceLms.landmark):
#                 #         if ind in feature:
#                 #             x, y, z = int(img_size[1]*lm.x), int(img_size[0]*lm.y), lm.z
#                 #
#                 #             img = cv2.circle(img, (x, y), 1, (0,0,255), 3)
#                 #             if curr_past == 0:
#                 #                 last_landmarks[cid][index] = abs(x) + abs(y) + abs(z)
#                 #             elif curr_past % 1 == 0:
#                 #                 current_landmarks[cid][index] = abs(x) + abs(y) + abs(z)
#                 #             index += 1
#                 #             color_ind += 1
#
#
#
#             # lips coordinates
#             lx1, ly1 = int(faceLms.landmark[61].x*img_size[1]), int(faceLms.landmark[61].y*img_size[0])
#             lx2, ly2 = int(faceLms.landmark[291].x*img_size[1]), int(faceLms.landmark[291].y*img_size[0])
#             lx3, ly3 = int(faceLms.landmark[0].x*img_size[1]), int(faceLms.landmark[0].y*img_size[0])
#             lx4, ly4 = int(faceLms.landmark[17].x*img_size[1]), int(faceLms.landmark[17].y*img_size[0])
#
#             # left eye coordinates
#             x1, y1 = int(faceLms.landmark[133].x*img_size[1]), int(faceLms.landmark[133].y*img_size[0])
#             x2, y2 = int(faceLms.landmark[33].x*img_size[1]), int(faceLms.landmark[33].y*img_size[0])
#             x4, y4 = int(faceLms.landmark[159].x*img_size[1]), int(faceLms.landmark[159].y*img_size[0])
#             x3, y3 = int(faceLms.landmark[145].x*img_size[1]), int(faceLms.landmark[145].y*img_size[0])
#
#             # lip horizontal-vertial LINE
#             img = cv2.line(img, (lx1, ly1), (lx2, ly2), (255,0,255), 2)
#             img = cv2.line(img, (lx3, ly3), (lx4, ly4), (255,0,0), 2)
#
#             # eye horizontal-vertial LINE
#             img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 2)
#             img = cv2.line(img, (x3,y3), (x4,y4), (0,0,255), 2)
#
#             #screen axes
#             img = cv2.line(img, (600,0), (600,720), (255,255,255), 1)
#             img = cv2.line(img, (0,350), (1200,350), (255,255,255), 1)
#
#
#             # lip euclidean ratio
#             lipa, lipb = ear(lx1, ly1, lx2, ly2, lx3, ly3, lx4, ly4)
#             lip_euclidean = lipa + lipb
#             # eye euclidean ratio
#             eyea, eyeb = ear(x1,y1,x2,y2,x3,y3,x4,y4)
#             euclidean = eyea/eyeb
#
#             # print(x1,y1,x2,y2,x3,y3,x4,y4)
#             #full=1 flinch=2, half=3, closed=4
#             #full=2 half=1 closed=0
#
#             if curr_past == 0:
#                 last_euclidean_distance_list[cid] = euclidean_distance
#                 last_facial_change = lip_euclidean
#             else:
#                 current_euclidean_distance_list[cid] = euclidean_distance
#                 current_facial_change = lip_euclidean
#
#
#             if euclidean > 0.29 and (current_eye_state == 0 or current_eye_state == 1):#fully open
#                 blink_count += 1
#                 current_eye_state = 2
#                 state_string = "FULL OPEN"
#             elif euclidean < 0.15 and (current_eye_state == 2 or current_eye_state == 1):
#                 blink_count += 1
#                 current_eye_state = 0
#                 state_string = "FULL CLOSED"
#             elif 0.18 <= euclidean < 0.29 and (current_eye_state == 0 or current_eye_state == 2):
#                 blink_count += 1
#                 current_eye_state = 1
#                 state_string = "HALF OPEN/CLOSED"
#
#             img = cv2.putText(img, f"{state_string}", (1000,80), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
#
#             img = cv2.putText(img, f"EAR : {euclidean}", (20, 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2 )
#
#
#             for i in range(7):
#                 # diff_list[i] = (sum((abs(last_landmarks-current_landmarks))[i]))/feature_landmarks_count[i]
#                 # if i == 2:
#                 # pass
#                 diff_list[i] = (sum((np.power(abs(last_landmarks-current_landmarks), 1))[i]))/feature_landmarks_count[i]
#                 # diff_list[i] = (sum((np.power(abs(last_landmarks-current_landmarks), 1))[i]))
#
#             frame = curr_past #current past frame
#             # eye = diff_list[0] if diff_list[0] > 1 else 1
#             zoop = (current_euclidean_distance_list[5]/feature_landmarks_count[5]) - (last_euclidean_distance_list[5]/feature_landmarks_count[5])
#             # eye = 0 if -5 < zoop < 5 else zoop
#             eye = 0
#             # facial_change = pow(5, facial_distance/1000)
#             facial_change = facial_distance
#             # eargraph = angle_vertical + angle_horizontal
#             eargraph = abs(last_facial_change - current_facial_change)
#             # eargraph = lip_euclidean
#             # print(eargraph)
#             # facial_change = (current_euclidean_distance_list[5]/feature_landmarks_count[5])/80
#             # eargraph =  2 if euclidean < 0.29 else (if)
#             # eargraph = current_eye_state
#
#             out_text = "HEAD MOVED" if(eye >= 5 or eye <= -5) else "---------"
#             cv2.putText(img, out_text, (1000,50), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
#             cv2.putText(img, str("Lip/Cheek : "+str(eargraph)), (1000,100), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
#
#             last_facial_change = current_facial_change
#             last_landmarks = deepcopy(current_landmarks)
#
#             if curr_past % 10 == 0:
#                 last_euclidean_distance_list = deepcopy(current_euclidean_distance_list)
#             curr_past += 1
#
#         else:
#             # print("Result in results.multi_face_landmarks failed. Fail Count : ", fail_count);
#             fail_count += 1
#         cTime = time.time()
#         fps = 1 / (cTime - pTime)
#         pTime = cTime
#         cv2.putText(img, f"fps: {int(fps)}", (20, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
#         cv2.imshow("image", img)
#         if cv2.waitKey(1) & 0xFF == ord('x'):
#             break
#
#
# cap.release()
# cv2.destroyAllWindows()